In [8]:
import xarray as xr
from datetime import datetime as dt
import numpy as np

In [9]:
date_i_want = dt(2022, 6, 2)

In [10]:
sbf_obs = xr.open_dataset(f'/Volumes/LtgSSD/tobac_Saves/tobac_Save_{date_i_want.strftime("%Y%m%d")}/seabreeze-obs.zarr')
for old_name in ['min_L2-MCMIPC', 'max_reflectivity']:
    new_name = 'feature_'+old_name
    new_name = new_name.replace('-', '_')
    ren = sbf_obs[old_name].rename(new_name)
    sbf_obs[new_name] = ren
    sbf_obs = sbf_obs.drop_vars(old_name)

/var/folders/sp/1j_lh9313dg42jrzhsfb4mk40000gn/T/ipykernel_16163/827038159.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  sbf_obs = xr.open_dataset(f'/Volumes/LtgSSD/tobac_Saves/tobac_Save_{date_i_want.strftime("%Y%m%d")}/seabreeze-obs.zarr')


In [11]:
sbf_obs

<xarray.Dataset> Size: 6GB
Dimensions:                            (feature: 16210, time: 283,
                                        vertical_levels: 2000, y: 1001,
                                        x: 1001, track: 770, cell: 945)
Coordinates:
  * track                              (track) float64 6kB 0.0 1.0 ... 769.0
  * time                               (time) datetime64[ns] 2kB 2022-06-02T0...
  * cell                               (cell) int64 8kB -1 4 6 ... 5107 5110
  * feature                            (feature) int64 130kB 1 2 ... 16209 16210
  * vertical_levels                    (vertical_levels) int64 16kB 0 1 ... 1999
  * x                                  (x) float64 8kB -2.5e+05 ... 2.5e+05
  * y                                  (y) float64 8kB -2.5e+05 ... 2.5e+05
Data variables: (12/93)
    feature_rhvdeficitcol              (feature) float32 65kB ...
    feature_time_index                 (feature) int64 130kB ...
    feature_flash_count_area_GT_4km    (feature) int64 130kB ...
    continental_sfc_rh                 (time) float64 2kB ...
    feature_lon                        (feature) float64 130kB ...
    maritime_msl_profile               (time, vertical_levels) float64 5MB ...
    ...                                 ...
    feature_parent_cell_id             (feature) int64 130kB ...
    maritime_v_profile                 (time, vertical_levels) float64 5MB ...
    feature_kdpwt_total                (feature) float64 130kB ...
    track_child_cell_count             (track) float64 6kB ...
    feature_min_L2_MCMIPC              (feature) float64 130kB ...
    feature_max_reflectivity           (feature) float64 130kB ...
Attributes:
    center_lat:      29.47190094
    center_lon:      -95.07873535
    soundings_used:  ['HOU_20220602_0030.csv', 'HOU_20220602_0150.csv', 'HOU_...

In [12]:
time_only_vars = [dv for dv in sbf_obs.data_vars if ('time',) == sbf_obs[dv].dims and dv != 'closest_satellite_time']
time_only = sbf_obs[time_only_vars].reset_coords(drop=True)
vars_to_reproc = np.unique([dv.replace('continental', '').replace('maritime', '') for dv in time_only_vars])
time_only_features = time_only.isel(time=sbf_obs.feature_time_index.data)
time_only_features = time_only_features.assign(
    feature = ('time', sbf_obs.feature.data)
)
time_only_features = time_only_features.swap_dims({'time' : 'feature'}).drop_vars('time')
for dv in vars_to_reproc:
    print(dv)
    continental_var = time_only_features[f'continental{dv}']
    maritime_var = time_only_features[f'maritime{dv}']
    feature_var = xr.where(sbf_obs.feature_seabreeze == -2, continental_var, xr.where(sbf_obs.feature_seabreeze == -1, maritime_var, np.nan))
    feature_var = feature_var.rename(f'feature{dv}')
    time_only_features[f'feature{dv}'] = feature_var
    time_only_features = time_only_features.drop_vars([f'continental{dv}', f'maritime{dv}'])
sbf_obs = xr.merge([sbf_obs, time_only_features])

_ccl
_convT
_el
_lcl
_lfc
_ll_rh
_mlcape
_mlcin
_mlecape
_sfc_rh
_six_km_bwd
_six_km_lapse


In [13]:
feature_only_vars = [dv for dv in sbf_obs.data_vars if 'feature' in sbf_obs[dv].dims]
vlevels_only_vars = [dv for dv in sbf_obs.data_vars if 'vertical_levels' in sbf_obs[dv].dims]

In [14]:
feature_only = sbf_obs[feature_only_vars].reset_coords(drop=True).drop_vars('feature_maxrefl')
vlevels_only = sbf_obs[vlevels_only_vars].reset_coords(drop=True)

In [15]:
regrouped = feature_only.to_pandas().groupby(['feature_parent_track_id', 'feature_time_index']).agg({
    'feature_area': np.nansum,
    'feature_ccl' : np.nanmean,
    'feature_convT' : np.nanmean,
    'feature_echotop' : np.nanmax,
    'feature_el' : np.nanmean,
    'feature_lcl' : np.nanmean,
    'feature_lfc' : np.nanmean,
    'feature_ll_rh' : np.nanmean,
    'feature_flash_count': np.nansum,
    'feature_flash_count_area_GT_4km': np.nansum,
    'feature_flash_count_area_LE_4km': np.nansum,
    'feature_flash_density': np.nansum,
    'feature_flash_density_area_GT_4km' : np.nansum,
    'feature_flash_density_area_LE_4km' : np.nansum,
    'feature_grid_cell_count' : np.nansum,
    'feature_hdim1_coordinate' : np.nanmean,
    'feature_hdim2_coordinate' : np.nanmean,
    'feature_kdpcol' : np.nanmax,
    'feature_kdpcol_mean' : np.nanmean,
    'feature_kdpcol_total' : np.nansum,
    'feature_kdpwt_total' : np.nansum,
    'feature_lat' : np.nanmean,
    'feature_lon' : np.nanmean,
    'feature_max_reflectivity' : np.nanmax,
    'feature_min_L2_MCMIPC': np.nanmin,
    'feature_mlcape' : np.nanmean,
    'feature_mlcin' : np.nanmean,
    'feature_mlecape' : np.nanmean,
    'feature_projection_x_coordinate' : np.nanmean,
    'feature_projection_y_coordinate' : np.nanmean,
    'feature_rhvdeficitcol' : np.nanmax,
    'feature_rhvdeficitcol_mean' : np.nanmean,
    'feature_rhvdeficitcol_total' : np.nansum,
    'feature_rhvdeficitwt_total' : np.nansum,
    'feature_seabreeze' : np.nanmean,
    'feature_sfc_rh' : np.nanmean,
    'feature_six_km_bwd' : np.nanmean,
    'feature_six_km_lapse' : np.nanmean,
    'feature_zdrcol' : np.nanmax,
    'feature_zdrcol_mean' : np.nanmean,
    'feature_zdrcol_total' : np.nansum,
    'feature_zdrwt_total' : np.nansum
}, engine='numba', engine_kwargs={'nopython' : True, 'parallel': True})
regrouped.columns = [col.replace('feature', 'track') for col in regrouped.columns.values]
regrouped = regrouped.to_xarray()
regrouped = regrouped.reindex(feature_parent_track_id=sbf_obs.track.data, fill_value=np.nan).rename({'feature_parent_track_id' : 'track'})
regrouped['track'] = regrouped['track'].astype(int)
regrouped['time'] = sbf_obs.time[regrouped.feature_time_index]
regrouped = regrouped.swap_dims({'feature_time_index': 'time'}).reset_coords('feature_time_index', drop=True)
sbf_obs = xr.merge([regrouped, sbf_obs])

/var/folders/sp/1j_lh9313dg42jrzhsfb4mk40000gn/T/ipykernel_16163/1989824296.py:1: FutureWarning: The provided callable <function nansum at 0x117206d40> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  regrouped = feature_only.to_pandas().groupby(['feature_parent_track_id', 'feature_time_index']).agg({
/var/folders/sp/1j_lh9313dg42jrzhsfb4mk40000gn/T/ipykernel_16163/1989824296.py:1: FutureWarning: The provided callable <function nanmean at 0x117207240> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  regrouped = feature_only.to_pandas().groupby(['feature_parent_track_id', 'feature_time_index']).agg({
/var/folders/sp/1j_lh9313dg42jrzhsfb4mk40000gn/T/ipykernel_16163/1989824296.py:1: FutureWarning: The provided callable <function nanmax at 0x1172

In [16]:
vlevels_feature = vlevels_only.isel(time=sbf_obs.feature_time_index.data)
vlevels_feature = vlevels_feature.assign(feature=(('time'), sbf_obs.feature.data)).swap_dims({'time' : 'feature'}).drop_vars('time')
vars_to_reproc = np.unique([dv.replace('maritime', '').replace('continental', '') for dv in list(vlevels_feature.data_vars)]).tolist()
for dv in vars_to_reproc:
    maritime_var = vlevels_feature[f'maritime{dv}']
    continental_var = vlevels_feature[f'continental{dv}']
    vlevels_feature[f'feature{dv}'] = xr.where(sbf_obs.feature_seabreeze == -2, continental_var, xr.where(sbf_obs.feature_seabreeze == -1, maritime_var, np.nan))
    vlevels_feature = vlevels_feature.drop_vars([maritime_var.name, continental_var.name])

In [17]:
vlevels_feature = vlevels_feature.assign_coords(feature_parent_track_id=sbf_obs.feature_parent_track_id)
vlevels_feature = vlevels_feature.assign_coords(feature_time_index=sbf_obs.feature_time_index)

In [18]:
df = vlevels_feature.to_dataframe()
grouped = df.groupby(['feature_parent_track_id', 'feature_time_index', 'vertical_levels']).mean()
grouped = grouped.to_xarray().rename({'feature_parent_track_id' : 'track'})
grouped['time'] = sbf_obs.time.isel(time=grouped.feature_time_index)
grouped = grouped.swap_dims({'feature_time_index': 'time'}).reset_coords('feature_time_index', drop=True).reindex(track=sbf_obs.track.data, fill_value=np.nan)
rename_dict = {dv : dv.replace('feature', 'track') for dv in grouped.data_vars}
grouped = grouped.rename(rename_dict)

In [19]:
sbf_obs = xr.merge([sbf_obs, grouped])

In [20]:
sbf_obs

<xarray.Dataset> Size: 34GB
Dimensions:                            (track: 770, time: 283, cell: 945,
                                        feature: 16210, vertical_levels: 2000,
                                        x: 1001, y: 1001)
Coordinates:
  * track                              (track) int64 6kB 0 1 2 3 ... 767 768 769
  * time                               (time) datetime64[ns] 2kB 2022-06-02T0...
  * cell                               (cell) int64 8kB -1 4 6 ... 5107 5110
  * feature                            (feature) int64 130kB 1 2 ... 16209 16210
  * vertical_levels                    (vertical_levels) int64 16kB 0 1 ... 1999
  * x                                  (x) float64 8kB -2.5e+05 ... 2.5e+05
  * y                                  (y) float64 8kB -2.5e+05 ... 2.5e+05
Data variables: (12/155)
    track_area                         (track, time) float64 2MB nan ... 211.0
    track_ccl                          (track, time) float64 2MB nan ... 884.1
    track_convT                        (track, time) float64 2MB nan ... 32.69
    track_echotop                      (track, time) float64 2MB nan ... 6e+03
    track_el                           (track, time) float64 2MB nan ... 145.1
    track_lcl                          (track, time) float64 2MB nan ... 900.1
    ...                                 ...
    track_dewpoint_profile             (track, time, vertical_levels) float64 3GB ...
    track_msl_profile                  (track, time, vertical_levels) float64 3GB ...
    track_pressure_profile             (track, time, vertical_levels) float64 3GB ...
    track_temperature_profile          (track, time, vertical_levels) float64 3GB ...
    track_u_profile                    (track, time, vertical_levels) float64 3GB ...
    track_v_profile                    (track, time, vertical_levels) float64 3GB ...